In [67]:
# Imports

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import random

from sklearn.utils import check_random_state


In [68]:
"""# Set random seed for NumPy
np.random.seed(42)
# Set random seed for Python's built-in random module
random.seed(42)
# Set global seed for scikit-learn's check_random_state (if applicable)
check_random_state(42)"""

"# Set random seed for NumPy\nnp.random.seed(42)\n# Set random seed for Python's built-in random module\nrandom.seed(42)\n# Set global seed for scikit-learn's check_random_state (if applicable)\ncheck_random_state(42)"

# Data analisys

In [69]:
activities_df=pd.read_csv("../data/activitats.csv",  encoding="ISO-8859-1")
marks_df=pd.read_csv("../data/notes.csv", sep=";")
uploads_df=pd.read_csv("../data/trameses.csv")


In [70]:
activities_df.head()

,activitat_id,activitat,aula_id,startdate,duedate,grade
0,3,Problema 1.1 - Hello world!!!,87,0,0,100
1,4,Problema 1.2 - Hello world!!! ++,87,0,0,100
2,5,Problema 3a.1: Càlcul edat,87,0,0,100
3,6,Problema 3a.8: Conversió dòlars a euros,87,0,0,100
4,7,Problema 3a.11: Mitjana de notes,87,0,0,100


In [71]:
activities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1801 entries, 0 to 1800
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   activitat_id  1801 non-null   int64 
 1   activitat     1801 non-null   object
 2   aula_id       1801 non-null   int64 
 3   startdate     1801 non-null   int64 
 4   duedate       1801 non-null   int64 
 5   grade         1801 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 84.6+ KB


In [72]:
marks_df.head()

,userid,aula_id,P_Grade,P_Grade_Date,F_Grade,F_Grade_Date,R_Grade,R_Grade_Date
0,176,92,NaN,NaN,"9,25",1667829600,NaN,1675173600
1,179,92,NaN,NaN,"7,75",1667829600,NaN,1675173600
2,168,92,NaN,NaN,"7,75",1667829600,NaN,1675173600
3,401,92,NaN,NaN,"3,50",1667829600,"8,25",1675173600
4,482,92,NaN,NaN,"7,00",1667829600,NaN,1675173600


In [73]:
uploads_df.head()

,id,activitat_id,userid,datesubmitted,grader,dategraded,grade,nevaluations
0,6,3,31,1658829978,0.0,1.658830e+09,100.0,1
1,7,362,26,1658837049,0.0,1.658837e+09,100.0,0
2,9,362,26,1658837075,0.0,1.658837e+09,100.0,0
3,15,362,31,1662537577,0.0,1.662538e+09,NaN,0
4,16,362,31,1662537593,0.0,1.662712e+09,100.0,0


In [74]:
from pandasql import sqldf
print(sqldf("SELECT * from uploads_df WHERE id=7"))

   id  activitat_id  userid  datesubmitted  grader    dategraded  grade  \
0   7           362      26     1658837049     0.0  1.658837e+09  100.0   

   nevaluations  
0             0  


## Data cleaning

### Delete unnecesary variables

In [75]:
# Dates are unnecessary information for our goal
activities_df = activities_df.drop('startdate', axis=1)
activities_df = activities_df.drop('duedate', axis=1)

marks_df = marks_df.drop('P_Grade_Date', axis=1)
marks_df = marks_df.drop('F_Grade_Date', axis=1)
marks_df = marks_df.drop('R_Grade_Date', axis=1)

uploads_df = uploads_df.drop('datesubmitted', axis=1)
uploads_df = uploads_df.drop('dategraded', axis=1)

#uploads_df = uploads_df.drop('grader', axis=1) #Also a useles column


### Delete missing data

In [76]:
print(activities_df.isnull().sum())
print()
print(marks_df.isnull().sum())
print()
print(uploads_df.isnull().sum())


activitat_id    0
activitat       0
aula_id         0
grade           0
dtype: int64

userid       0
aula_id      0
P_Grade    867
F_Grade    381
R_Grade    838
dtype: int64

id                   0
activitat_id         0
userid               0
grader               7
grade           253347
nevaluations         0
dtype: int64


In [77]:
# We delete the column recuperation
marks_df = marks_df.drop('R_Grade', axis=1)

### Add new variables

In [78]:
# We create a new boolean variable to ask wheter the student has made the partial exam or not
marks_df['made_partial'] = marks_df['P_Grade'].isnull()
marks_df = marks_df.drop('P_Grade', axis=1) # We drop this column as it is not useful anymore

### Filling missing values

In [80]:
uploads_df['grader'].fillna(uploads_df['grader'].median(), inplace=True)

uploads_df['grade'].fillna(0, inplace=True)

## Data preprocessing